<a href="https://colab.research.google.com/github/techn-0/Machine_Learning_STD/blob/main/6_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 as cv
import numpy as np
from PyQt5.QtWidgets import *
import sys
import winsound

class TrafficWeak(QMainWindow):
  def __init__(self):
    super().__init__()
    self.setWindowTitle('영화찾기')
    self.setGeometry(300, 300, 800, 300)

    signButton = QPushButton('포스터 등록', self)
    roadButton = QPushButton('영화관 영상 불러오기', self)
    recognitionButton = QPushButton('인식', self)
    quitButton = QPushButton('종료', self)
    self.label = QLabel('welcome',self)

    signButton.setGeometry(10, 10, 100, 30)
    roadButton.setGeometry(110, 10, 100, 30)
    recognitionButton.setGeometry(210, 10, 100, 30)
    quitButton.setGeometry(510, 10, 100, 30)
    self.label.setGeometry(10, 50, 780, 250)

    signButton.clicked.connect(self.signFunction)
    roadButton.clicked.connect(self.roadFunction)
    recognitionButton.clicked.connect(self.recognitionFunction)
    quitButton.clicked.connect(self.quitFunction)

    self.signFiles = [['토토로.jpg', '토토로'], ['로보캅.jpg', '로보캅'], ['메트로폴리스.jpg', '매트로폴리스']]
    self.signImgs = []

  def signFunction(self):
    self.label.clear()
    self.label.setText('포스터 등록')

    for fname, _ in self.signFiles:
      self.signImgs.append(cv.imread(fname))
      cv.imshow(fname, self.signImgs[-1])

  def roadFunction(self):
    if self.signImgs==[]:
      self.label.setText('먼저 포스터 등록!')
    else:
      fname = QFileDialog.getOpenFileName(self, 'Open file', './')
      self.roadImg = cv.imread(fname[0])
      if self.roadImg is None: sys.exit('파일 찾을 수 없음')

      cv.imshow('Road scene', self.roadImg)

  def recognitionFunction(self):
    if self.roadImg is None:
      self.label.setText('먼저 영상을 입력!')
    else:
      sift = cv.SIFT_create()

      KD = []
      for img in self.signImgs:
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        KD.append(sift.detectAndCompute(gray, None))


      grayRoad = cv.cvtColor(self.roadImg, cv.COLOR_BGR2GRAY) # 명암 변환
      road_kp, road_des = sift.detectAndCompute(gray, None) # 키포인트, 기술자 추출

      matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
      GM = []

      for sign_kp, sign_des in KD:
        knn_matches = matcher.knnMatch(sign_des, road_des, 2)
        T = 0.7
        good_match = []
        for nearest1, nearest2 in knn_matches:
          if (nearest1.distance / nearest2.distance) < T:
            good_match.append(nearest1)
        GM.append(good_match)

      best = GM.index(max(GM, key = len)) # 매칭 쌍 최대 표지판 찾기
      if len(GM[best]) < 4: # 4개 미만 시 실패
        self.label.setText('목표가 없습니다.')
      else: #성공하면 출력
        sign_kp = KD[best][0]
        good_match = GM[best]

        points1 = np.float32([sign_kp[gm.queryIdx].pt for gm in good_match])
        points2 = np.float32([road_kp[gm.trainIdx].pt for gm in good_match])

        H, _ = cv.findHomography(points1, points2, cv.RANSAC)

        h1, w1 = self.signImgs[best].shape[0], self.signImgs[best].shape[1]
        h2, w2 = self.roadImg.shape[0], self.roadImg.shape[1]

        box1 = np.float32([[0, 0], [0, h1-1], [w1-1, h1-1], [w1-1, 0]]).reshape(4, 1, 2)
        box2 = cv.perspectiveTransform(box1, H)

        self.roadImg = cv.polylines(self.roadImg, [np.int32(box2)], True, (0, 255, 0), 4)

        img_match = np.empty((max(h1, h2), w1+w2, 3), dtype=np.uint8)
        cv.drawMatches(self.signImgs[best], sign_kp, self.roadImg, road_kp, good_match, img_match, flags = cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        cv.imshow('Matches and Homography', img_match)

        self.label.setText(self.signFiles[best][1] + '영화 입니다.')
        winsound.Beep(2000, 500)

  def quitFunction(self):
    cv.destroyAllWindows()
    self.close()

app = QApplication(sys.argv)
window = TrafficWeak()
window.show()
app.exec_()

